# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


# Intergrax — Engine Planner (Planner-only Test Notebook)

## Purpose

This notebook is dedicated to **isolated testing and iterative improvement of the Intergrax `EnginePlanner`**, completely **independent from the engine execution pipeline** (`DropInKnowledgeRuntime.run()`).

It is designed to:

- run `EnginePlanner.plan(...)` for various user prompts,
- inspect and compare generated `EnginePlan` outputs,
- evaluate plan stability, correctness, and feasibility,
- iteratively refine the planner prompt, plan schema, and validation rules,
- improve the planner safely without impacting the runtime `run()` behavior.

## Scope

This notebook:

- does **not** call `DropInKnowledgeRuntime.run()`,
- does **not** execute RAG / tools / websearch / attachments retrieval,
- does **not** generate a final user answer,
- focuses exclusively on **plan generation and analysis**.

## Architectural assumptions

- `EnginePlanner` is part of the runtime, but it must be testable in isolation.
- Planning is **LLM-driven** (no decision heuristics in code).
- Availability of mechanisms (RAG, websearch, tools, attachments, user LTM) is provided explicitly as **capabilities**.
- `EnginePlan` is a strongly-typed artifact that will later drive execution in a **Planner → Executor → Verifier** architecture.

## How to use

1. Initialize the same LLM adapter and `RuntimeConfig` you use in the runtime.
2. Create a minimal `RuntimeState` containing the fields required by the planner (`base_history`, `final_system_instructions`, `cap_*`).
3. Run the planner for different prompts.
4. Inspect the resulting `EnginePlan` and its `debug` field.
5. Iterate until plans are stable, consistent, and feasible.

This notebook is the first step toward converting the engine into a true **ChatGPT-like** runtime:
**Planner → Executor → Verifier**.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from intergrax.llm.messages import ChatMessage
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.planning.engine_planner import EnginePlanner

# -------------------------------------------------------
# Global test constants
# -------------------------------------------------------

USER_ID = "demo-user-planner"
SESSION_ID = "sess_planner_only_001"

# -------------------------------------------------------
# Build LLM adapter and RuntimeConfig
# IMPORTANT: Replace build_llm_adapter() with your actual builder.
# This must be the same adapter/config used in your runtime.
# -------------------------------------------------------

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="auto",
    enable_user_longterm_memory=True,    
)

planner = EnginePlanner(llm_adapter=llm_adapter)

async def _sanity_check_llm():
    try:
        msgs = [
            ChatMessage(role="system", content="Return exactly: OK"),
            ChatMessage(role="user", content="Say OK"),
        ]
        out = llm_adapter.generate_messages(msgs, run_id="planner_sanity")
        # handle async adapters too
        import inspect
        if inspect.iscoroutine(out):
            out = await out
        print("LLM sanity output:", (out or "")[:200])
    except Exception as e:
        print("LLM sanity check failed:", str(e))

await _sanity_check_llm()


LLM sanity output: OK


## Minimal RuntimeState for planner-only tests

The planner requires a minimal subset of runtime context:

- `RuntimeRequest` (user_id, session_id, message, attachments)
- `RuntimeState` fields used by the planner:
  - `base_history` (recent messages, optional)
  - `final_system_instructions` (optional)
  - capability flags: `cap_rag_available`, `cap_user_ltm_available`, `cap_attachments_available`, `cap_websearch_available`, `cap_tools_available`

In this notebook we will set capability flags explicitly,
so we can test multiple scenarios deterministically.


In [3]:
from intergrax.llm_adapters.llm_usage_track import LLMUsageTracker
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_state import RuntimeState
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


def build_planner_request(
    *,
    user_id: str,
    session_id: str,
    message: str,
    instructions: str | None = None,
    attachments: list | None = None,
) -> RuntimeRequest:
    return RuntimeRequest(
        user_id=user_id,
        session_id=session_id,
        message=message,
        attachments=attachments or [],
        instructions=instructions,
    )


def build_planner_state(
    *,
    req: RuntimeRequest,
    run_id: str,
) -> RuntimeState:
    """
    Minimal RuntimeState for planner-only tests.
    Uses only existing RuntimeState fields (no extra structures).
    """
    state = RuntimeState(
        request=req,
        run_id=run_id,
        llm_usage_tracker=LLMUsageTracker(run_id=run_id),
    )

    # Keep it runtime-like: register the same adapter label as core runtime does.
    state.llm_usage_tracker.register_adapter(config.llm_adapter, label="core_adapter")

    # Minimal history (optional; can be empty)
    state.base_history = [
        ChatMessage(role="user", content="We are testing the Intergrax planner."),
        ChatMessage(role="assistant", content="Understood."),
    ]

    # Optional: simulate memory-layer instruction fragments (if needed later)
    state.profile_user_instructions = None
    state.profile_org_instructions = None

    # Capabilities — explicitly set for planner tests (no heuristics, no reflection)
    state.cap_rag_available = bool(config.enable_rag)
    state.cap_user_ltm_available = bool(config.enable_user_longterm_memory)
    state.cap_attachments_available = True   # planner-only: we can toggle later per test
    state.cap_websearch_available = bool(config.enable_websearch)
    state.cap_tools_available = bool(config.tools_mode != "off")

    return state


async def run_planner(
    *,
    message: str,
    instructions: str | None = "Use a technical, concise style. Never use emojis in code blocks or technical documentation.",
) :
    req = build_planner_request(
        user_id=USER_ID,
        session_id=SESSION_ID,
        message=message,
        instructions=instructions,
    )

    state = build_planner_state(req=req, run_id="planner-test-run-001")

    plan = await planner.plan(req=req, state=state, config=config, run_id=state.run_id)

    print("\n=== ENGINE PLAN ===")
    plan.print_pretty()

    return plan


In [4]:
await run_planner(
    message="Explain how to implement an async retry strategy in Python for API calls."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'planner_json_len': 754,
           'planner_raw_len': 754,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": '
                                  '"implement_async_retry_strategy_python_api_calls",\n'
                                  '  "reasoning_summary": "Answer is based on '
                                  'general Python async patterns; no external '
                                  'sources needed.",\n'
                                  '  "ask_clarifying_question": false,\n'
                                  '  "clarifying_question": null,\n'
                                  '  "use_rag": {\n'
                                  '    "enabled": false,\n'
                                  '    "top_k": 0,\n'
                                  '    "max_chars": 0\n'
                        

EnginePlan(version='1.0', intent='implement_async_retry_strategy_python_api_calls', reasoning_summary='Answer is based on general Python async patterns; no external sources needed.', ask_clarifying_question=False, clarifying_question=None, use_rag=PlanRetrieval(enabled=False, top_k=0, max_chars=0), use_user_longterm_memory=PlanRetrieval(enabled=False, top_k=0, max_chars=0), use_attachments=PlanRetrieval(enabled=False, top_k=0, max_chars=0), use_websearch=PlanRetrieval(enabled=False, top_k=0, max_chars=0), use_tools=PlanTools(enabled=False, goal='', tool_choice_hint=None), parallel_groups=[], final_answer_style='concise_technical', debug={'planner_raw_len': 754, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "implement_async_retry_strategy_python_api_calls",\n  "reasoning_summary": "Answer is based on general Python async patterns; no external sources needed.",\n  "ask_clarifying_question": false,\n  "clarifying_question": null,\n  "use_rag": {\n    "enabled": false,\n    "